In [88]:
import os
header = {"Authorization": "Bearer "+os.getenv("ECOPORTAL_TOKEN"),
          "Content-Type": "application/json"}

In [89]:
import os
import requests

header = {"Authorization": "Bearer "+os.getenv("ECOPORTAL_TOKEN"),
          "Content-Type": "application/json"}

api_url = ' https://data.eco-platform.org/resource/processes?search=true&distributed=true&virtual=true&metaDataOnly=false&validUntil=2022&format=JSON'

response = requests.get(url=api_url, headers=header)
main = response.json()

for x in range(2, 11):
    response = requests.get(url=api_url, headers=header)
    results = response.json()
    
    num = x
    baseURL = 'https://epdnorway.lca-data.com/resource/datastocks/91413340-7bf0-4f88-a952-0f91cba685df/processes/'
    uuid = main['data'][num]['uuid']
    version = main['data'][num]['version']
    api_url = f'{baseURL}{uuid}?version={version}&format=JSON'

    response = requests.get(url=api_url, headers=header)
    results = response.json()

    name = results['processInformation']['dataSetInformation']['name']['baseName'][0]['value']
    print(name)

 ABb PMB, Skanska Industrial Solutions, Dalby asfaltverk
 ABb, Skanska Industrial Solutions, Dalby asfaltverk
 AG, Skanska Industrial Solutions, Dalby asfaltverk
 AG, Skanska Industrial Solutions, Luleå Asfaltverk
 AXLJ-RMF Eco Cable 3x240/35 LT 12kV
 Betonelement Grå væg
 Betonelement Væg C35 Passiv
 Bewi Finland Oy EPS Lattia
 Birsta S N2 W4 c/c4  m


In [1]:
import os
import requests

header = {"Authorization": "Bearer "+os.getenv("ECOPORTAL_TOKEN"),
          "Content-Type": "application/json"}

api_url = ' https://data.eco-platform.org/resource/processes?search=true&distributed=true&virtual=true&metaDataOnly=false&validUntil=2022&format=JSON'

response = requests.get(url=api_url, headers=header)
main = response.json()

In [2]:
import json

baseURL = 'https://epdnorway.lca-data.com/resource/datastocks/91413340-7bf0-4f88-a952-0f91cba685df/processes/'
baseURL2 = 'https://ibudata.lca-data.com/resource/datastocks/7f92c48a-07c6-4a0c-91fd-4166e6138402/processes/'
baseURL3 = 'https://node.epditaly.it/resource/processes/'
num = 10

uuid = main['data'][num]['uuid']
version = main['data'][num]['version']
api_url2 = f'{baseURL}{uuid}?version={version}&format=JSON'
print(api_url2)
response2 = requests.get(url=api_url2, headers=header)
results2 = response2.json()

https://epdnorway.lca-data.com/resource/datastocks/91413340-7bf0-4f88-a952-0f91cba685df/processes/7f73caec-8a20-41d2-9b7d-16c368bfe27e?version=00.03.000&format=JSON


In [3]:
def find_value(d, value, path=""):
    found = []

    if isinstance(d, dict):
        for key, dvalue in d.items():
            new_path = f"{path}.{key}" if path else key
            
            if isinstance(dvalue, str) and value in dvalue: #added for partial match
                found.append(new_path)
            elif isinstance(dvalue, (dict, list)):
                found.extend(find_value(dvalue, value, new_path))

    elif isinstance(d, list):
        for index, item in enumerate(d):
            new_path = f"{path}[{index}]"

            if isinstance(item, str) and value in item:
                found.append(new_path)  #partial
            elif isinstance(item, (dict, list)):
                found.extend(find_value(item, value, new_path))

    return found

search = "GWP-total"
dPath = find_value(results2, search)

print(dPath)

['LCIAResults.LCIAResult[1].referenceToLCIAMethodDataSet.shortDescription[0].value']


In [ ]:
main['data'][2]

In [2]:
def find_value(d, value, path=""):
    found = []

    if isinstance(d, dict):
        for key, dvalue in d.items():
            new_path = f"{path}.{key}" if path else key
            
            if isinstance(dvalue, str) and value in dvalue: #added for partial match
                found.append(new_path)
            elif isinstance(dvalue, (dict, list)):
                found.extend(find_value(dvalue, value, new_path))

    elif isinstance(d, list):
        for index, item in enumerate(d):
            new_path = f"{path}[{index}]"

            if isinstance(item, str) and value in item:
                found.append(new_path)  #partial
            elif isinstance(item, (dict, list)):
                found.extend(find_value(item, value, new_path))

    return found

In [3]:
def get_nested_value(d, keys, default='Value Not Found'):

    for key in keys:
        try:
            d = d[key]
        except (KeyError, IndexError, TypeError):
            return default
    return d

In [20]:
import re

def extract(path, type):
    if not path:
        return None
    
    if type == 1:
        pattern = r"LCIAResults\.LCIAResult\[(\d+)\]"
        match = re.search(pattern, path)
        
        if match:
            return int(match.group(1))
        else:
            return None
        
    elif type == 2:
        #pattern = r"anies\[(\d+)\]"
        pattern = r"\[(\d+)\]\.module"
        match = re.search(pattern, path)

        if match:
            return int(match.group(1))
        else:
            return None
    
    elif type != 1 and type != 2:
        print('Expected type to be 1 or 2')

In [33]:
def assign(path, type): #this type is for the function to know whether A1-A3 are bundled or not, 0 is false and 1 is true
    if type == 0:
        A2 = path[extract(find_value(path, 'A2')[0], 2)]['value'] if find_value(path, 'A4') else 'variable not found'
        A3 = path[extract(find_value(path, 'A3')[0], 2)]['value'] if find_value(path, 'A4') else 'variable not found'
        A4 = path[extract(find_value(path, 'A4')[0], 2)]['value'] if find_value(path, 'A4') else 'variable not found'
        A5 = path[extract(find_value(path, 'A5')[0], 2)]['value'] if find_value(path, 'A5') else 'variable not found'
        
        C1 = path[extract(find_value(path, 'C1')[0], 2)]['value'] if find_value(path, 'C1') else 'variable not found'
        C2 = path[extract(find_value(path, 'C2')[0], 2)]['value'] if find_value(path, 'C2') else 'variable not found'
        C3 = path[extract(find_value(path, 'C3')[0], 2)]['value'] if find_value(path, 'C3') else 'variable not found'
        C4 = path[extract(find_value(path, 'C4')[0], 2)]['value'] if find_value(path, 'C4') else 'variable not found'

        D = path[extract(find_value(path, 'D')[0], 2)]['value'] if find_value(path, 'D') else 'variable not found'
        D1 = path[extract(find_value(path, 'D1')[0], 2)]['value'] if find_value(path, 'D1') else 'variable not found'
        D2 = path[extract(find_value(path, 'D2')[0], 2)]['value'] if find_value(path, 'D2') else 'variable not found'

        return A2, A3, A4, A5, C1, C2, C3, C4, D, D1, D2
    
    elif type == 1:
        A4 = path[extract(find_value(path, 'A4')[0], 2)]['value'] if find_value(path, 'A4') else 'variable not found'
        A5 = path[extract(find_value(path, 'A5')[0], 2)]['value'] if find_value(path, 'A5') else 'variable not found'
        
        C1 = path[extract(find_value(path, 'C1')[0], 2)]['value'] if find_value(path, 'C1') else 'variable not found'
        C2 = path[extract(find_value(path, 'C2')[0], 2)]['value'] if find_value(path, 'C2') else 'variable not found'
        C3 = path[extract(find_value(path, 'C3')[0], 2)]['value'] if find_value(path, 'C3') else 'variable not found'
        C4 = path[extract(find_value(path, 'C4')[0], 2)]['value'] if find_value(path, 'C4') else 'variable not found'

        D = path[extract(find_value(path, 'D')[0], 2)]['value'] if find_value(path, 'D') else 'variable not found'
        D1 = path[extract(find_value(path, 'D1')[0], 2)]['value'] if find_value(path, 'D1') else 'variable not found'
        D2 = path[extract(find_value(path, 'D2')[0], 2)]['value'] if find_value(path, 'D2') else 'variable not found'

        return A4, A5, C1, C2, C3, C4, D, D1, D2

In [85]:
import re

category = ['GWP-total','GWP-biogenic','GWP-fossil','GWP-IOBC']
#category = ['GWP-total','GWP-total','GWP-total','GWP-total']

EPDs = []
none = "Unmapped"
path1 = 0
path2 = 0
i = 0

for x in range(0,36):
    i += 1 
    done = 0
    num = x
    uuid = main['data'][num]['uuid']
    version = main['data'][num]['version']
    uri = main['data'][num]['uri']
    api_url = f'{uri}&format=JSON'
    #api_url = f'{baseURL}{uuid}?version={version}&format=JSON'

    response = requests.get(url=api_url, headers=header)

    try:
        results = response.json()
    except requests.exceptions.JSONDecodeError as e:
        out = f'json #{num}: Failed to parse JSON. Error: {e}'
        print(out)
        print(f'Raw response text: {response.text[:200]}')
        EPDs.append(out)
        continue



    ############################### DATA PATHS #################################
    EPDname = results['processInformation']['dataSetInformation']['name']['baseName'][0]['value']
    
    desc1 = get_nested_value(results, ['processInformation', 'technology', 'technologicalApplicability', 0, 'value'])
    technology_desc = get_nested_value(results, ['processInformation', 'technology', 'technologyDescriptionAndIncludedProcesses', 0, 'value'])
    desc = f"{desc1} — {technology_desc}" if desc1 and technology_desc else 'No Secondary Description'


    #comm = get_nested_value(results['processInformation']['dataSetInformation']['generalComment'][0]['value'])
    #valid = get_nested_value(results['processInformation']['time']['timeRepresentativenessDescription'][0]['value'])

    comm = get_nested_value(results, ['processInformation', 'dataSetInformation', 'generalComment', 0, 'value'])
    valid = get_nested_value(results, ['processInformation', 'time', 'timeRepresentativenessDescription', 0, 'value'])
    

    #Determine formatting style with find function   

    if find_value(results, 'GWP-total'):
        type = 1
    elif find_value(results, 'Global warming potential (GWP)'):
        type = 2
        print('type 2')
        EPDs.append(type)
        continue
    else:
        type = 'Unknown'
        print('unrecognized format')
        EPDs.append(type)
        continue
    
    
    path = results['LCIAResults']['LCIAResult'][path1]['other']['anies']


    if type == 1:

        for x in range(0,4):
            find = category[x]
            path1 = extract(find_value(results, find)[0], 1)
            #print(category[x])
            if find_value(path, 'A1-A3'):
                #print('Bundled')
                path2 = extract(find_value(path, 'A1-A3')[0], 2)
                #A11 = A12 = A13 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                if x == 0:
                    A11 = A12 = A13 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A14, A15, C11, C12, C13, C14, D1, D11, D12 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)
                elif x == 1:
                    A21 = A22 = A23 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A24, A25, C21, C22, C23, C24, D2, D21, D22 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)
                elif x == 2:
                    A31 = A32 = A33 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A34, A35, C31, C32, C33, C34, D3, D31, D32 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)
                elif x == 3:
                    A41 = A42 = A43 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A44, A45, C41, C42, C43, C44, D4, D41, D42 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 1)


            elif find_value(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 'A1'):
                #print('Individual')
                path2 = extract(find_value(path, 'A1')[0], 2)
                #A11 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                #A12, A13, A14, A15, C11, C12, C13, C14, D1, D11, D12 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                if x == 0:
                    A11 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A12, A13, A14, A15, C11, C12, C13, C14, D1, D11, D12 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                elif x == 1:
                    A21 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A22, A23, A24, A25, C21, C22, C23, C24, D2, D21, D22 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                elif x == 2:
                    A31 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A32, A33, A34, A35, C31, C32, C33, C34, D3, D31, D32 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)
                elif x == 3:
                    A41 = results['LCIAResults']['LCIAResult'][path1]['other']['anies'][path2]['value']
                    A42, A43, A44, A45, C41, C42, C43, C44, D4, D41, D42 = assign(results['LCIAResults']['LCIAResult'][path1]['other']['anies'], 0)


            else:
                print('error')

    #A = results['LCIAResults']['LCIAResult'][gwpT]['other']['anies'][0]['value']


    material_data = {
        "material_id": uuid,
        "display_name": EPDname,
        "source": none,
        "description": f'{desc},',
        "comments": comm,
        "validity": valid,
        "thickness": {
            "qty": none,
            "unit": none
        },
        "service_life": none,
        "waste_rate": none,
        "transportation": {
            "mode": none,
            "distance": {
                "qty": none,
                "unit": none
            }
        },
        "density": {
            "qty": none,
            "unit": none
        },
        "linear_density": {
            "qty": none,
            "unit": none
        },
        "declared_unit": none,
        "lcia_method": none,
        "background_database": none,
        "impacts": {
            "gwp_total": {
                "A1": A11, "A2": A12, "A3": A13, "A4": A14, "A5": A15, "C1": C11,
                "C2": C12, "C3": C13, "C4": C14, "D": D1, "D1": D11, "D2": D12
            },
            "gwp_biogenic": {
                "A1": A21, "A2": A22, "A3": A23, "A4": A24, "A5": A25, "C1": C21,
                "C2": C22, "C3": C23, "C4": C24, "D": D2, "D1": D21, "D2": D22
            },
            "gwp_fossil": {
                "A1": A31, "A2": A32, "A3": A33, "A4": A34, "A5": A35, "C1": C31,
                "C2": C32, "C3": C33, "C4": C34, "D": D3, "D1": D31, "D2": D32
            },
            "gwp_iobc": {
                "A1": A41, "A2": A42, "A3": A43, "A4": A44, "A5": A45, "C1": C41,
                "C2": C42, "C3": C43, "C4": C44, "D": D4, "D1": D41, "D2": D42
            }
        },
        "characteristics": {}
    }
    
    EPDs.append(material_data)
    print(f'json #{i} is done')



json #1 is done
json #3 is done
json #4 is done
json #5 is done
json #6 is done
json #7 is done
json #8 is done
json #9 is done
json #10 is done
json #11 is done


IndexError: list index out of range

In [80]:
EPDs[4]

{'material_id': '982ea073-82cb-4515-aa4f-b2f7381105af',
 'display_name': ' AG, Skanska Industrial Solutions, Dalby asfaltverk',
 'source': 'Unmapped',
 'description': 'Hot mix base course for roads AG — AG base course according to Swedish road administration specification TDOK 2013:0529,',
 'comments': 'Data quality:\nSpecific data for the product composition are provided by the manufacturer. The data represent the production of the declared product and were collected for EPD development in the year of study. Background data is based on EPDs according to EN 15804 and different LCA databases. The data quality of the raw materials in A1 is presented in the table below. eurobitume (2019) is not considered conservative according to EN 15804, but is used due to common practice in other LCA tools, EPDs and PCR.\nSpecific environmental data from EPDs (Skanska EPD), in accordance with EN 15804, have been used for aggregates. Similarly, specific data have been used for transport distances from 

In [84]:
import re

EPDs = []
none = "Unmapped"
log = []

for x in range(0,14):
    num = x
    uuid = main['data'][num]['uuid']
    version = main['data'][num]['version']
    uri = main['data'][num]['uri']
    api_url = f'{uri}&format=JSON'
    #api_url = f'{baseURL}{uuid}?version={version}&format=JSON'

    response = requests.get(url=api_url, headers=header)

    try:
        results = response.json()
    except requests.exceptions.JSONDecodeError as e:
        out = f'json #{num}: Failed to parse JSON. Error: {e}'
        #print(f'Raw response text: {response.text[:200]}')  # Show a preview of the raw response for debugging
        continue  # Skip to the next iteration

    if find_value(results, 'GWP-total'):
        out = f'json #{num}: type 1'
    elif find_value(results, 'Global warming potential (GWP)'):
        out = f'json #{num}: type 2'
    else:
        out = f'json #{num}: other'

    print(out)
    log.append(out)

json #0: type 1
json #1: type 2
json #2: type 1
json #3: type 1
json #4: type 1
json #5: type 1
json #6: type 1
json #7: type 1
json #8: type 1
json #9: type 1
json #10: type 1
json #11: type 1
json #12: type 2
json #13: type 1
